In [1]:
import numpy as np
import torch
from Embeddings import ComplexPreference, train_on_func, evaluate
import sys
sys.path.append('../')
from TorchUtilityFunctions import ackley, branin, eggholder, matyas, michalewicz, rosenbrock, bukin

In [2]:
#Default parameters used by the functions in their implementation. Should bypass the need for re-writing
class AffineTransform:
    def __init__(self):
        self.affine_transform = np.array([1., 0.])
params = AffineTransform()

In [3]:
funcs = {
    "ackley": ackley,
    "branin": branin,
    "eggholder": eggholder,
    "matyas": matyas,
    "michalewicz": michalewicz,
    "rosenbrock": rosenbrock,
    "bukin": bukin,
}

In [5]:
factor = 2
sizes = [256, 128, 64]
for name, func in funcs.items():
    model = ComplexPreference(2, sizes=sizes)
    print("-"*20)
    train_on_func(model, func, epochs=100, lr=0.01, patience=25)
    evaluate(model, func)
    torch.save(model.state_dict, f'../Embedding_Model_Weights/{name}.pt')

In [7]:
#Meant to save the models and not the state_dicts, correcting here
for name, _ in funcs.items():
    model = torch.jit.load(f"../Embedding_Model_Weights/{name}.pt", map_location="cpu")
    cplex_pref = ComplexPreference(2, sizes=sizes)
    cplex_pref.load_state_dict(model.state_dict())
    torch.save({
        "hparams": cplex_pref.hparams,
        "state_dict": cplex_pref.state_dict(),
    }, f'../Embedding_Model_Weights/{name}.pt')